Importing libraries

In [12]:
!echo !pip install matplotlib

!pip install matplotlib
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)


In [1]:
import numpy
import tensorflow
import matplotlib
import pandas as pd

ModuleNotFoundError: No module named 'tensorflow'